In [7]:
import sys
import json
import numpy as np
import random
import time
sys.path.append('../utils/')
from utils import gpt_decoder, load_json, load_jsonl
def write(fp, line):
    with open(fp, 'a', encoding='utf-8') as f:
        f.write(json.dumps(line) + '\n')

In [2]:
l1_datas = load_jsonl('l1_timebench.jsonl')
l1_datas_50 = l1_datas[:50]
l1_datas_10 = l1_datas_50[:10]

In [3]:
prompt_100 = "{}?"
prompt_101 = "{}? The answer is"
prompt_102 = "{}? Directly output the answer, don't output the reasoning process. The answer is"
prompt_200 = "{}? Let's think step by step."

answer_trigger = "{} Therefore, the answer is"

In [8]:
def run(pattern, datas):
    results = []
    for data in datas:
        q = data['question']
        a = data['answer']
        prompt = pattern.format(q)
        predict = gpt_decoder(prompt, n=1)
        results.append([q, a, predict])
        write('./l1_results.jsonl', {'q':q, 'a':a[0], 'p':predict[0]})
        time.sleep(1.0)
    return results



In [9]:
results_100 = run(prompt_100, l1_datas_10)

time consume :11.522489547729492
time consume :6.4037439823150635
time consume :7.552268743515015
time consume :2.7674190998077393
time consume :7.861821413040161
time consume :7.366655349731445
time consume :6.492549896240234
time consume :2.3944826126098633
time consume :2.064324378967285
time consume :8.125284671783447


In [ ]:
results_101 = run(prompt_101, l1_datas_10)
results_102 = run(prompt_102, l1_datas_10)
results_200 = run(prompt_200, l1_datas_10)

In [17]:
results_200 = run(prompt_200, l1_datas_10[-2:])

time consume :15.69370985031128
time consume :11.54982042312622


In [41]:
import re
re.sub("\(|\)|:|\.|\|\n|\,", "", "Therefore, 1 year and 1 month after May, 1111 is June,   1112.")
patter = r"(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Sept|Oct|Nov|Dec|January|February|March|April|May|June|July|August|September|October|November|December)\s*([0-9]{2,4})"


In [46]:
pred =  "Therefore, 1 year and 1 month aft."
pred = pred.strip()
import re
pred = re.sub("\(|\)|:|\.|\|\n|\,", "", pred)
results = re.findall(patter, pred)
results

[]

In [94]:
def extract_date(text):
    if isinstance(text, list):
        text = text[0]
    pattern = r"(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Sept|Oct|Nov|Dec|January|February|March|April|May|June|July|August|September|October|November|December)\s*([0-9]{2,4})"
    text = text.strip()
    text = re.sub("\(|\)|:|\.|\|\n|\,", "", text)
    results = re.findall(pattern, text)
    if len(results) > 0:
        return list(results[-1])
    else:
        return None
MONTH_TO_NUM = {
    'Jan' : 1, 'Feb' : 2, 'Mar' : 3, 'Apr' : 4, 'May' : 5, 'Jun' : 6, 'Jul' : 7, 'Aug' : 8, 'Sep' : 9, 'Sept' : 9, 'Oct' : 10, 'Nov' : 11, 'Dec' : 12,
    'January' : 1, 'February' : 2, 'March' : 3, 'April' : 4, 'May' : 5, 'June' : 6, 'July' : 7, 'August' : 8, 'September' : 9, 'October' : 10, 'November' : 11 ,'December' : 12 }
def is_date_equal(date1, date2):
    #Date format (Month, Year)
    def convert_date_to_list(date):
        if isinstance(date, list) and len(date) == 1:
            date = date[0]
        if isinstance(date, str):
            date = date.split(",")
            date = [d.strip() for d in date]
        date = [MONTH_TO_NUM[date[0].lower().capitalize()], int(date[1])]
        return date
    date1, date2 = convert_date_to_list(date1), convert_date_to_list(date2)
    if date1[0] != date2[0]:
        return False
    if date1[1] != date2[1]:
        return False
    return True

In [96]:
pred = []
false_pred = []
for res in [*results_100, *results_101, *results_102, *results_200]:
    a = res[1]
    p = res[2]
    p = extract_date(p)
    res = is_date_equal(a, p)
    if res is True:
        pred.append(1)
    else:
        pred.append(0)

np.mean(pred[0:10]), np.mean(pred[10:20]), np.mean(pred[20:30]), np.mean(pred[30:40])

(0.7, 0.9, 0.7, 0.5)

In [83]:
isinstance(a, str)

True

In [84]:
a.split(',')[0]

'May'